In [1]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [56]:
yr = 2025
datadir = joinpath("..", "Data", string(yr))

"../Data/2025"

In [57]:
seasondata = DataFrame(CSV.File(joinpath(datadir, "dataset_with_odds_and_performance.csv")))

Row,pitches,player_id,player_name,game_date,game_pk,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,pitcher_run_exp,run_exp,bat_speed,swing_length,pa,bip,singles,doubles,triples,hrs,so,k_percent,bb,bb_percent,api_break_z_with_gravity,api_break_z_induced,api_break_x_arm,api_break_x_batter_in,hyper_speed,bbdist,hardhit_percent,barrels_per_bbe_percent,barrels_per_pa_percent,release_pos_z,release_pos_x,plate_x,plate_z,obp,barrels_total,batter_run_value_per_100,xobp,xslg,pitcher_run_value_per_100,xbadiff,xobpdiff,xslgdiff,wobadiff,swing_miss_percent,arm_angle,attack_angle,attack_direction,swing_path_tilt,rate_ideal_attack_angle,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,HomeID,AwayID,HomeRuns,AwayRuns,join_key,spitcher_id,spitcher_name,winner,offensive_xwoba,offensive_launch_angle,offensive_launch_speed,offensive_hardhit_percent,last5_o_hardhit_percent,last5_o_launch_angle,last5_o_launch_speed,last5_o_xwoba,last_p_hardhit_percent,last_p_launch_angle,last_p_launch_speed,last_p_xwoba,home_opener_odds,away_opener_odds
,Int64,Int64,String3,Date,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Int64?,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64,Float64?,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,String31,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String7?,String7?
1,169,108,LAA,2025-09-28,776135,169,100.0,0.297,0.351,0.292,0.649,0.44,0.386,0.262,11,37,88.5,8.4,2235,87.0,86.68,21,72,97,-0.3,6.11,107,150,121,149,311,329,302,-6.488,6.488,71.0,7.3,43,28,6,1,0,4,9,20.9,4,9.3,2.45183,0.61012,0.32775,-0.16266,92.1,165,39.2857,14.2857,9.30233,5.78,0.59,-0.19,2.3,0.395,4,3.83905,0.365,0.525,-3.83905,0.035,0.03,0.124,0.054,29.2,33.3,10.0807,1.04779,33.2773,0.4,37.2089,31.0858,108,117,2,6,2025092810811726,691951,"Aldegheri, Sam",false,0.298,13.5,89.5,44.0,45.8349,19.1,89.7,0.2974,27.2727,7.5,85.1,0.359,200,-10000
2,146,117,HOU,2025-09-28,776135,146,100.0,0.176,0.088,0.208,0.265,0.218,0.298,0.238,6,34,89.5,13.5,2210,87.4,87.47,21,75,71,0.1,6.3,102,153,119,146,316,331,292,2.343,-2.343,70.7,7.3,36,25,5,0,0,1,9,25.0,2,5.6,2.43233,0.59,0.2674,-0.0211,91.8,161,44.0,4.0,2.77778,5.41,-0.02,0.17,2.32,0.222,1,-1.60479,0.281,0.406,1.60479,-0.062,-0.059,-0.141,-0.08,28.0,29.3,8.92424,-0.552705,34.1227,0.5,36.7464,29.759,108,117,2,6,2025092810811726,621121,"McCullers Jr., Lance",true,0.386,8.4,88.5,39.2857,33.3423,17.02,86.64,0.256,44.3584,14.9083,89.5917,0.314333,200,-10000
3,174,135,SD,2025-09-28,776136,174,100.0,0.297,0.162,0.385,0.459,0.359,0.316,0.266,11,37,86.9,7.8,2231,89.5,89.16,18,76,98,-0.3,6.29,108,150,122,148,298,317,295,-1.262,1.262,69.0,7.2,41,27,7,3,0,1,10,24.4,4,9.8,2.06592,0.84529,0.38144,-0.12213,93.2,159,44.4444,3.7037,2.43902,5.26,0.23,0.07,2.45,0.366,1,0.725287,0.337,0.37,-0.725287,0.031,0.029,0.089,0.043,23.7,32.0,9.82242,-3.65927,31.5219,0.506667,35.1397,32.2173,135,109,12,4,20250928135109124,676664,"Sears, JP",true,0.372,9.8,92.1,51.8519,43.6603,10.5,88.76,0.3376,31.0231,17.775,84.75,0.29975,-117,111
4,171,109,AZ,2025-09-28,776136,172,99.4,0.314,0.2,0.385,0.514,0.386,0.372,0.305,11,35,92.1,9.8,2423,89.1,88.77,17,87,84,-0.3,6.31,113,144,120,147,288,316,283,-2.921,2.921,69.3,7.2,41,27,6,4,0,1,9,22.0,4,9.8,2.41292,0.

In [58]:
allgames = unique(seasondata.game_pk)

cnames = ["game_pk", "game_date", "home_odds", "home_id", "home_name", "home_winner", "home_last5_o_hardhit",
        "home_last5_o_launch_speed", "home_last5_o_launch_angle", "home_last5_o_xwoba",
        "home_last_p_hardhit", "home_last_p_launch_speed", "home_last_p_launch_angle", 
        "home_last_p_xwoba", "away_odds", "away_id", "away_name", "away_winner", "away_last5_o_hardhit",
        "away_last5_o_launch_speed", "away_last5_o_launch_angle", "away_last5_o_xwoba",
        "away_last_p_hardhit", "away_last_p_launch_speed", "away_last_p_launch_angle", 
        "away_last_p_xwoba"]

finalset = DataFrame([name => [] for name in cnames])

usefulcols = [:player_id, :player_name, :winner, :last5_o_hardhit_percent,
                :last5_o_launch_speed, :last5_o_launch_angle, :last5_o_xwoba, 
                :last_p_hardhit_percent, :last_p_launch_speed, :last_p_launch_angle,
                :last_p_xwoba]

for game = allgames
    # Only want to do this if it's clearly a match
    if (size(seasondata[seasondata.game_pk .== game, :], 1) == 2)
    
        towork = seasondata[seasondata.game_pk .== game, :]
        row = []
        push!(row, game)
        push!(row, towork[1, :game_date])
        push!(row, towork[1, :home_opener_odds])

        if towork[1, :HomeID] == towork[1, :player_id]
            order = [1, 2]
        else
            order = [2, 1]
        end

        homerow = towork[order[1], usefulcols]
        [push!(row, a) for a in Array(homerow)]
        # push!(row, Array(homerow))
        push!(row, towork[1, :away_opener_odds])
        awayrow = towork[order[2], usefulcols]
        [push!(row, a) for a in Array(awayrow)]
        # push!(row, Array(awayrow))

        push!(finalset, row)
    end
end

In [59]:
CSV.write(joinpath(datadir, "final_format.csv"), finalset)

"../Data/2025/final_format.csv"